In [1]:
# %load plot_template.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

plt.rc('font', family='serif')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10, color='grey')
plt.rc('ytick', labelsize=10, color='grey')
plt.rc('legend', fontsize=12, loc='lower left')
plt.rc('figure', titlesize=12)
plt.rc('savefig', dpi=330, bbox='tight')
%matplotlib inline

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPRegressor

In [5]:
# Replace PATH_TO_ROOT

import sys
PATH_TO_ROOT = '/Users/guanyuxiaoxiong/City-Scale-Electricity-Use-Prediction-master'
sys.path.insert(0,PATH_TO_ROOT)

In [18]:
def train_test_split(data_array, train_ratio=0.75):
    n_train = int(len(data_array)*train_ratio)
    data_train = data_array[:n_train,:]
    data_test = data_array[n_train:,:]
    return data_train, data_test

def generate_fig_path(fig_name):
    '''
    Generate the figure path and name
    To be used by plt.savefig function
    '''
    return '{}/results/fig/section3.4 tabular data model/{}'.format(PATH_TO_ROOT, fig_name)

def generate_input_data_path(data_name):
    '''
    Generate the path to input data
    '''
    return '{}/results/data/{}.csv'.format(PATH_TO_ROOT, data_name)

def generate_result_data_path(data_name):
    '''
    Generate the path to save the result data
    '''
    return '{}/results/data/predict/{}.csv'.format(PATH_TO_ROOT, data_name)

In [9]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def generate_result_data_path(filename):
    result_dir = '/home/aistudio/results'
    os.makedirs(result_dir, exist_ok=True)
    return os.path.join(result_dir, filename)

def otherML_train(region):
    # 定义文件路径
    file_path = f'/home/aistudio/data/{region}_hourly_modified.xlsx'

    # 读取数据
    data = pd.read_excel(file_path, index_col=0)
    data.index = pd.to_datetime(data.index, dayfirst=True)
    data.sort_index(inplace=True)
    data = data.truncate(after='2019-07-01')
# 添加 Holiday 列：周末设为假日（1），工作日为 0
    data['Holiday'] = (data.index.weekday >= 5).astype(int)

    # 添加滞后负荷特征和滑动平均特征
    data['load_lag1'] = data['load'].shift(1)
    data['load_avg3'] = data['load'].rolling(window=3).mean()

 

      # 保留需要的列并去除缺失
    data = data[['load', 'Temperature', 'Holiday', 'load_lag1', 'load_avg3']].dropna()

    # 特征拼接：目标值 + 特征
    data_ml = np.concatenate((
        data[['load', 'Temperature', 'load_lag1', 'load_avg3']].values,
        data[['Holiday']].astype('int').values,
        pd.get_dummies(data.index.weekday).values,
        pd.get_dummies(data.index.month).values
    ), axis=1)


    # 划分训练测试集
    train_size = int(len(data_ml) * 0.75)
    data_train = data_ml[:train_size]
    data_test = data_ml[train_size:]

    X_train = data_train[:, 1:]
    y_train = data_train[:, 0].ravel()
    X_test = data_test[:, 1:]
    y_test = data_test[:, 0].ravel()
    X_all = np.concatenate((X_train, X_test), axis=0)

    # 训练 Random Forest 模型
    regr_rf = RandomForestRegressor(max_depth=8, min_samples_leaf=5, max_features='sqrt')
    regr_rf.fit(X_train, y_train)
    yhat_train_rf = regr_rf.predict(X_train)
    yhat_test_rf = regr_rf.predict(X_test)
    rmse_train_rf = mean_squared_error(y_train, yhat_train_rf, squared=False)
    rmse_test_rf = mean_squared_error(y_test, yhat_test_rf, squared=False)
    cvrmse_train_rf = rmse_train_rf / np.mean(y_train)
    cvrmse_test_rf = rmse_test_rf / np.mean(y_test)
    print(f'-------Random Forest: {region}--------------')
    print(f'RMSE for Train: {rmse_train_rf}')
    print(f'RMSE for Test: {rmse_test_rf}')
    print(f'CVRMSE for Train: {cvrmse_train_rf:.4f}')
    print(f'CVRMSE for Test: {cvrmse_test_rf:.4f}')

    # 训练 SVM 模型
    regr_svr = make_pipeline(StandardScaler(), SVR(C=10.0, epsilon=0.5))
    regr_svr.fit(X_train, y_train)
    yhat_train_svr = regr_svr.predict(X_train)
    yhat_test_svr = regr_svr.predict(X_test)
    rmse_train_svr = mean_squared_error(y_train, yhat_train_svr, squared=False)
    rmse_test_svr = mean_squared_error(y_test, yhat_test_svr, squared=False)
    cvrmse_train_svr = rmse_train_svr / np.mean(y_train)
    cvrmse_test_svr = rmse_test_svr / np.mean(y_test)
    print(f'-------SVM: {region}--------------')
    print(f'RMSE for Train: {rmse_train_svr}')
    print(f'RMSE for Test: {rmse_test_svr}')
    print(f'CVRMSE for Train: {cvrmse_train_svr:.4f}')
    print(f'CVRMSE for Test: {cvrmse_test_svr:.4f}')

    # 训练神经网络模型
    regr_nn = MLPRegressor(hidden_layer_sizes=(32, 32, 16), activation='relu',
                           solver='adam', alpha=0.001, max_iter=500,
                           learning_rate='adaptive', learning_rate_init=0.01, shuffle=True)
    regr_nn.fit(X_train, y_train)
    yhat_train_nn = regr_nn.predict(X_train)
    yhat_test_nn = regr_nn.predict(X_test)
    rmse_train_nn = mean_squared_error(y_train, yhat_train_nn, squared=False)
    rmse_test_nn = mean_squared_error(y_test, yhat_test_nn, squared=False)
    cvrmse_train_nn = rmse_train_nn / np.mean(y_train)
    cvrmse_test_nn = rmse_test_nn / np.mean(y_test)
    print(f'-------Neural Network: {region}--------------')
    print(f'RMSE for Train: {rmse_train_nn}')
    print(f'RMSE for Test: {rmse_test_nn}')
    print(f'CVRMSE for Train: {cvrmse_train_nn:.4f}')
    print(f'CVRMSE for Test: {cvrmse_test_nn:.4f}')

    # 保存预测结果
    data['yhat_rf'] = regr_rf.predict(X_all)
    data['yhat_svm'] = regr_svr.predict(X_all)
    data['yhat_nn'] = regr_nn.predict(X_all)
    data.to_csv(generate_result_data_path(f'otherML_{region}.csv'))

    # 准备绘图数据
    verif_plot = data[['load', 'yhat_rf', 'yhat_svm', 'yhat_nn']].copy()
    verif_plot.rename(columns={'load': 'y'}, inplace=True)
    verif_plot['train'] = False
    verif_plot.iloc[:train_size, verif_plot.columns.get_loc('train')] = True

    return verif_plot


In [10]:
fuzhou_otherML = otherML_train('fuzhou')
sin_otherML = otherML_train('singapore')
bel_otherML = otherML_train('belgium')


-------Random Forest: fuzhou--------------
RMSE for Train: 674.0120229184553
RMSE for Test: 697.6518115987406
CVRMSE for Train: 0.0919
CVRMSE for Test: 0.0991
-------SVM: fuzhou--------------
RMSE for Train: 1222.467971249914
RMSE for Test: 1252.578039747163
CVRMSE for Train: 0.1667
CVRMSE for Test: 0.1780
-------Neural Network: fuzhou--------------
RMSE for Train: 678.150329763013
RMSE for Test: 666.5240458139724
CVRMSE for Train: 0.0925
CVRMSE for Test: 0.0947
-------Random Forest: singapore--------------
RMSE for Train: 175.41417201312134
RMSE for Test: 191.60006612226013
CVRMSE for Train: 0.0298
CVRMSE for Test: 0.0327
-------SVM: singapore--------------
RMSE for Train: 201.29143194090688
RMSE for Test: 416.68720247972163
CVRMSE for Train: 0.0342
CVRMSE for Test: 0.0712
-------Neural Network: singapore--------------
RMSE for Train: 184.87331907364685
RMSE for Test: 185.54340991891993
CVRMSE for Train: 0.0314
CVRMSE for Test: 0.0317
-------Random Forest: belgium--------------
RMSE f